In [6]:
from datetime import datetime
from haio import (
    HAIOClient,
    Gemini_IO,
    OpenAI_IO,
    Bedrock_IO,
    QuestionTemplate,
    img_to_url,
)
import json
import sys

sys.path.append("../modules")

from virtual_human_io import VirtualHuman_IO
from vec_to_img import vec_to_img

In [7]:
# prepare data

from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

labels = [
    "Airplane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Ship",
    "Truck",
]

In [8]:
# prepare haio


virtual_human_io = VirtualHuman_IO()
gemini_io = Gemini_IO()
openai_io = OpenAI_IO()
llama_io = Bedrock_IO("us.meta.llama3-2-90b-instruct-v1:0")
claude_io = Bedrock_IO("us.amazon.nova-lite-v1:0")

haio_client = HAIOClient(
    filepath="/Users/pictomo/Repositories/SequentialCTA_experiment/scripts",
    human_io=virtual_human_io,
    openai_io=openai_io,
    gemini_io=gemini_io,
    llama_io=llama_io,
    nova_io=claude_io,
    # claude_io=claude_io,
)

question_template: QuestionTemplate = QuestionTemplate(
    title="CIFAR-10 Image Classification",
    description="Classify the image using the CIFAR-10 dataset",
    question=[
        {"tag": "img", "src": 0},
        {
            "tag": "h1",
            "value": "Choose the label that best describes the image from the options.",
        },
        {"tag": "p", "value": " ".join(labels)},
    ],
    answer={
        "type": "select",
        "options": labels,
    },
)

In [9]:
async def expr1(execution_config: dict, task_count: int) -> dict:
    result = {"question_count": [0], "human_count": [0], "collect_count": [0]}
    for i in range(task_count):
        img_url = img_to_url(img_data=vec_to_img(x_train[i]), mime_type="image/png")
        data_list = [img_url]
        asked_questions = []
        asked_questions.append(
            haio_client.ask(
                question_template=question_template,
                data_list=data_list,
            )
        )
        answer_info: HAIOClient.MethodReturn = await haio_client.wait(
            asked_questions=asked_questions, execution_config=execution_config
        )
        result["question_count"].append(result["question_count"][-1] + 1)
        result["human_count"].append(
            result["human_count"][-1] + answer_info["add_human_assign"]
        )
        if answer_info["answer_list"][0] == labels[y_train[i][0]]:
            result["collect_count"].append(result["collect_count"][-1] + 1)

    return result

In [10]:
timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
execution_config = {"method": "sequential_cta_1", "quality_requirement": 0.9}
result = await expr1(execution_config=execution_config, task_count=10)
with open(f"../results/test{timestamp}.json", "w") as f:
    json.dump(result, f)